Импортируем библиотеку

In [1]:
import time

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn as nn
from transformers import BertModel
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import nltk
import pymorphy2
from nltk.corpus import stopwords
import re 
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 импортируем данные и уберем нулевые значения.

In [2]:
data = pd.read_csv('D:/1.Projects/1t/Attractor/model/nlp/project-2.csv', delimiter=',')

In [3]:
data

,sentiment,okpd2,by,number
0,Подходит,62.01.12.000: Услуги по проектированию и разра...,"Разработка Программы оперативного, сезонного и...",№32312762450
1,Скорее да,62.03.12.190: Услуги по управлению компьютерны...,Оказание услуг по сопровождению компонента «Ин...,№0187200001723001563
2,Подходит,62.03.12.190: Услуги по управлению компьютерны...,Оказание услуг по сопровождению компонента «И...,№0187200001723001575
3,Скорее да,62.01.11.000: Услуги по проектированию и разра...,Оказание услуг по модернизации подсистемы мони...,№0851200000623007196
4,Скорее да,62.01.12.000: Услуги по проектированию и разра...,Оказание информационных услуг в части поддержа...,№0345200016223000029
...,...,...,...,...
2003,Не подходит,62.02.30.000: Услуги по технической поддержке ...,Оказание услуг по сопровождению программы для ...,№32312640233
2004,Не подходит,62.02.30.000: Услуги по технической поддержке ...,на оказание услуги по обучению и консультирова...,№32312638416
2005,Не подходит,62.02.20.190: Услуги консультативные в области...,Оказанию услуг поддержки и сопровождения прогр...,№32312637404
2006,Не подходит,62.03.12.130: Услуги по сопровождению компьюте...,Консультационно-технологическое сопровождение ...,№32312637048


In [4]:
# data.rename(columns={'Реестровый номер закупки': 'reg_numb', 
#                      'Наименование закупки': 'name',
#                      'Классификация по ОКПД2': 'okpd2_class',
#                      'Дата размещения': 'date'}, inplace=True)

In [5]:
null_values = data.isnull().sum()
print(null_values)

sentiment    12
okpd2         1
by            0
number        0
dtype: int64


Удаляем строки с нулевыми значениями

In [6]:
data = data.dropna(subset=['sentiment'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1996 entries, 0 to 2007
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  1996 non-null   object
 1   okpd2      1995 non-null   object
 2   by         1996 non-null   object
 3   number     1996 non-null   object
dtypes: object(4)
memory usage: 78.0+ KB


In [8]:
le = LabelEncoder()
data['sentiment'] = le.fit_transform(data['sentiment'])
data['sentiment'] = data['sentiment'].astype(int)
data['number'] = data['number'].str.replace('№', '')

C:\Users\Mi\AppData\Local\Temp\ipykernel_7328\2883795514.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'] = le.fit_transform(data['sentiment'])
C:\Users\Mi\AppData\Local\Temp\ipykernel_7328\2883795514.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['sentiment'] = data['sentiment'].astype(int)
C:\Users\Mi\AppData\Local\Temp\ipykernel_7328\2883795514.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [9]:
data

,sentiment,okpd2,by,number
0,2,62.01.12.000: Услуги по проектированию и разра...,"Разработка Программы оперативного, сезонного и...",32312762450
1,3,62.03.12.190: Услуги по управлению компьютерны...,Оказание услуг по сопровождению компонента «Ин...,0187200001723001563
2,2,62.03.12.190: Услуги по управлению компьютерны...,Оказание услуг по сопровождению компонента «И...,0187200001723001575
3,3,62.01.11.000: Услуги по проектированию и разра...,Оказание услуг по модернизации подсистемы мони...,0851200000623007196
4,3,62.01.12.000: Услуги по проектированию и разра...,Оказание информационных услуг в части поддержа...,0345200016223000029
...,...,...,...,...
2003,1,62.02.30.000: Услуги по технической поддержке ...,Оказание услуг по сопровождению программы для ...,32312640233
2004,1,62.02.30.000: Услуги по технической поддержке ...,на оказание услуги по обучению и консультирова...,32312638416
2005,1,62.02.20.190: Услуги консультативные в области...,Оказанию услуг поддержки и сопровождения прогр...,32312637404
2006,1,62.03.12.130: Услуги по сопровождению компьюте...,Консультационно-технологическое сопровождение ...,32312637048


Сразу введем переменную, определяющую максимальное количество слов в предложении. \
Также скачаем словарь со стоп словами, создадим переменную с токенезацией от BERT и переменную с классом, который позволяет найти изначальную форму слова.

In [10]:
data.sentiment.nunique()

5

In [11]:
MAX_LEN=64

morph = pymorphy2.MorphAnalyzer()
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
# tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Проверим наличие GPU и создадим переменную с типом вычислений, которые вы используете. 

In [12]:
# Check device
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [13]:
# for i in data.sentiment:
#     if i == 'Подходит':
#         data.sentiment[data.sentiment == 'Подходит'] = 5
#     elif i == 'Скорее да':
#         data.sentiment[data.sentiment == 'Скорее да'] = 4
#     elif i == 'Скорее нет':
#         data.sentiment[data.sentiment == 'Скорее нет'] = 3
#     elif i == 'Затрудняюсь':
#         data.sentiment[data.sentiment == 'Затрудняюсь'] = 2
#     else:
#         data.sentiment[data.sentiment == 'Не подходит'] = 1

# data['sentiment'] = data['sentiment'].astype('int64')

Разделяем данные на train и val.

In [14]:
X = data.by.values
y = data.sentiment.values

X_train, X_val, y_train, y_val =\
    train_test_split(X, y, test_size=0.1, random_state=2020)

Создадим функцию которая очищает наши данные: приводит к нижнему регистру, удаляет стоп слова, удаляет различные лишние знаки и приводит слова к первоначальному значению.

In [15]:
def text_preprocessing(s):
    """
    - reduce to the subscript
    - delete "@name"
    - delete punctuation marks except "?"
    - delete other special characters
    - remove stop words
    - we bring all the words to the initial form
    """
    
    s = s.lower()
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?]', ' ', s)
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('russian')])
    s = " ".join([morph.parse(word)[0].normal_form for word in s.split()])
    s = re.sub(r'\s+', ' ', s).strip()
    
    return s

Напишем функцию которая подготавливает данные для модели. Тут мы используем переменную созданную в начале и укажем максимальное количество слов в предложении.  Также токенезируем все слова и создадим attention mask.

In [16]:
def preprocessing_for_bert(data, MAX_LEN):
    """
    Выполните необходимые шаги предварительной обработки для предварительно обученного BERT.
    @param    data (np.array): Массив текстов для обработки.
    @return   input_ids (torch.Tensor): Тензор идентификаторов токенов для передачи в модель.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                            tokens should be attended to by the model.
    """
    
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` будет:
        #   (1) Токенизировать предложение
        #   (2) Добавит токены `[CLS]` и `[SEP]` в начало и конец предложения
        #   (3) Усечет/расширит предложение до максимальной длины
        #   (4) Соотносит лексемы с их идентификаторами
        #   (5) Создаем маску внимания
        #   (6) Возвращает словарь выходов
        #   (7) Применяет специальный токен к индексу
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,             # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    return input_ids, attention_masks

Применим функцию для val и train.

In [17]:
train_inputs, train_masks = preprocessing_for_bert(X_train, MAX_LEN)
val_inputs, val_masks = preprocessing_for_bert(X_val, MAX_LEN)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Создаем Dataloader для загрузки val и train в модель.

In [18]:
# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 32

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

Создадим класс с моделью.

In [19]:
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 312, 32, 2

        # Instantiate BERT model
        # self.bert = AutoModel.from_pretrained("Den4ikAI/rubert_large_squad_2") # RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x768 and 312x32)
        self.bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

Добавим функцию, которая инициализирует модель.

In [20]:
def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

Создадим переменную с функцией потерь, функцию определяющую по какому алгоритму будут генерироваться случайные числа(это нужно для воспроизводимости модели в дальнейших тестах), функцию обучения модели и функцию считающую результаты на val.

In [21]:
# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)
    
    return val_loss, val_accuracy

Начнем обучение. Настоятельно рекомендую обучать на gpu, так как скорость по сравнению с cpu возрастает в разы.

In [22]:
import random
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

c:\Users\Mi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------


RuntimeError: expected scalar type Long but found Int

Напишем функцию для тестирования нашей модели.

In [ ]:
def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

Получим предсказание на val.

In [ ]:
probs = bert_predict(bert_classifier, val_dataloader)

Импортируем данные для test.

In [ ]:
орморм

In [ ]:
import csv

data = []
with open('D:/1.Projects/1t/Attractor/model/nlp/sent_analysis/data/Data.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file)
    for row in reader:
        if len(row) == 1: # проверяем, что в строке ожидаемое количество полей
            data.append(row)

df = pd.DataFrame(data[22500:45001])

In [ ]:
data = pd.read_csv('D:/1.Projects/1t/Attractor/model/nlp/sent_analysis/data/Data.csv', delimiter=',')[22500:45001] 

data = data.drop('Unnamed: 0', axis=1)
data = data.rename(columns={'Review': 'text'})
data = data.rename(columns={'Rating': 'sentiment'})
data = data.drop(data[data['text'].isnull() == True].index)

In [ ]:
test = pd.read_json('DataSet.json')[300000:301000].text #тут были взяты строчки с 300 000 по 301 000, так как модель обучалась на первых 100 000 строк.
test_y = pd.read_json('DataSet.json')[300000:301000].sentiment

Подготавливаем тестовые данные и создаем DataLoader

In [ ]:
# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test, MAX_LEN)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

Получаем результаты по тестовым данным и округляем вероятности принадлежности к классу.

In [ ]:
# Compute predicted probabilities on the test set
probs = bert_predict(bert_classifier, test_dataloader)

# Get predictions from the probabilities
threshold = 0.75
preds = np.where(probs[:, 1] > threshold, 1, 0)

Выводим получившиеся метрики.

In [ ]:
print("Test AUC: {:.3f}".format(roc_auc_score(test_y, preds)))
print("Test Recall: {:.3f}".format(recall_score(test_y, preds)))
print("Test Precision: {:.3f}".format(precision_score(test_y, preds)))
print("Test f1: {:.3f}".format(f1_score(test_y, preds)))